In [1]:
from matplotlib import pyplot as plt
import time
import datetime
from utils_laj import *
from data_processing import get_SensorData, my_pca, series_to_supervised, lstm_sampling
from sklearn.model_selection import train_test_split
import pandas as pd

today = datetime.date.today()
TR_END = "2017-12-31 23:00:00"

In [2]:
path_checkpoint = './save/save_lstm/lstm_2_layers'
input_path = './input_data'
file = os.path.join(input_path,'sensors.csv')
target_file = os.path.join(input_path,'coke_target.csv')

In [3]:
# Считываем данные из файлов, нормализуем и применяем метод главных комполнентов
X, y, submit_X, mean_y, std_y = get_SensorData(file, target_file, nc=12)


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Explained variance [0.5383315  0.09938994 0.08926971 0.04085576 0.03484106 0.032776
 0.02645778 0.01969716 0.01752129 0.01691421 0.01532406 0.01388007], Cumsum [0.5383315  0.63772144 0.72699115 0.76784692 0.80268798 0.83546398
 0.86192176 0.88161892 0.89914021 0.91605442 0.93137848 0.94525856]


In [4]:
# Размерности датасета с известным целевым y и полного датасета
print("Data X: %s, y %s, submit_X %s" % (X.shape, y.shape, submit_X.shape))

Data X: (10400, 12), y (10400, 2), submit_X (13272, 12)


In [5]:
# выделим последние записи в размере контрольного датасета в тестовый датасет
x1_train, x1_test, y1_train, y1_test = train_test_split(X, y, test_size=2920, random_state=42, shuffle=False)

# Формируем сэмплы для LSTM по 48 рядов внахлест с шагом 1
timesteps = 48
lag = 1
x_train, y_train = lstm_sampling(x1_train, y=y1_train['target'], timesteps=timesteps, lag=lag)
x_test, y_test = lstm_sampling(x1_test, y=y1_test['target'], timesteps=timesteps, lag=lag)
#yl = y['target'][timesteps:]

# формируем данные сабмита по аналогичному принципу + берем 47 шагов из тестовых данных
Xs, ys = lstm_sampling(submit_X[len(x_train)+len(x_test)+timesteps:], timesteps=timesteps, lag=lag)

print("Data x_train, y_train: (%s,%s);  x_test, y_test: (%s,%s); Xs, ys: (%s,%s) " % (x_train.shape, y_train.shape, x_test.shape, y_test.shape, Xs.shape, ys))



Samples length 7432
Targets length 7432
Samples length 2872
Targets length 2872
Samples length 2872
Targets length 0
Data x_train, y_train: ((7432, 48, 12),(7432, 48));  x_test, y_test: ((2872, 48, 12),(2872, 48)); Xs, ys: ((2872, 48, 12),None) 


In [6]:
    Train = True
    Predict = True
    plot = False

In [7]:
    batch_size = 359  # Batch size
    shift = batch_size
    #if Train == False: batch_size = 1

    sequence_length = timesteps  # Number of steps
    learning_rate = 4*10e-5  # 0.0001
    epochs = 1000
    ann_hidden = 16

    n_channels = x_train.shape[2]

    lstm_size = 48  # Number LSTM units
    num_layers = 3  # 2  # Number of layers
    alpha = 0 # regularization coef

In [8]:
    X = tf.placeholder(tf.float32, [None, sequence_length, n_channels], name='inputs')
    Y = tf.placeholder(tf.float32, [None, sequence_length], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    is_train = tf.placeholder(dtype=tf.bool, shape=None, name="is_train")

    conv_last_layer = X

    shape = conv_last_layer.get_shape().as_list()
    print('My Conv Shape:',shape)
    CNN_flat = tf.reshape(conv_last_layer, [-1, shape[1] * shape[2]])

    dence_layer_1 = dense_layer(CNN_flat, size=sequence_length * n_channels, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_1")
    lstm_input = tf.reshape(dence_layer_1, [-1, sequence_length, n_channels])

    cell = get_RNNCell(['LSTM'] * num_layers, keep_prob=keep_prob, state_size=lstm_size)
    init_state = cell.zero_state(batch_size, tf.float32)
    #init_state = tf.Variable(cell.zero_states(batch_size, tf.float32), trainable=False)
    rnn_output, states = tf.nn.dynamic_rnn(cell, lstm_input, dtype=tf.float32, initial_state=init_state)
    #with tf.control_dependencies([state.assign(states)]):
    #    output = tf.identity(output)
    
    stacked_rnn_output = tf.reshape(rnn_output, [-1, lstm_size])  # change the form into a tensor

    dence_layer_2 = dense_layer(stacked_rnn_output, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2")
    
    dence_layer_3 = dense_layer(dence_layer_2, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2_2")

    output = dense_layer(dence_layer_3, size=1, activation_fn=None, batch_norm=False, phase=is_train, drop_out=False,
                         keep_prob=keep_prob,
                         scope="fc_3_output")

    prediction = tf.reshape(output, [-1])
    y_flat = tf.reshape(Y, [-1])

    h = prediction - y_flat
    
    tv = tf.trainable_variables()
    regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])

    cost_function = tf.reduce_sum(tf.square(h)) + alpha*regularization_cost
    RMSE = tf.sqrt(tf.reduce_mean(tf.square(h)))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost_function)

    saver = tf.train.Saver()
    
    training_generator = batch_generator(x_train, y_train, batch_size, sequence_length, online=True, online_shift=shift)
    testing_generator = batch_generator(x_test, y_test, batch_size, sequence_length, online=True, online_shift=shift)
    #print(len(list(training_generator)))

    if Train: model_summary(learning_rate=learning_rate, batch_size=batch_size, lstm_layers=num_layers,
                            lstm_layer_size=lstm_size, fc_layer_size=ann_hidden, sequence_length=sequence_length,
                            n_channels=n_channels, path_checkpoint=path_checkpoint, spacial_note='')
        


My Conv Shape: [None, 48, 12]


In [ ]:
    with tf.Session() as session:
        tf.global_variables_initializer().run()

        if Train == True:
            #saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)

            cost = []
            plot_x = []
            plot_y1 = []
            plot_y2 = []
            iter_train = int(x_train.shape[0]/shift)
            iter_test = int(x_test.shape[0]/shift)
            print("Training set MSE")
            print("No epoches: ", epochs, "No itr: ", iter_train)
            __start = time.time()
            for ep in range(epochs):
                h1 = []
                t1 = []
                
                for itr in range(iter_train):
                    ## training ##
                    batch_x, batch_y = next(training_generator)
                    session.run(optimizer,
                                feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.6, learning_rate_: learning_rate})
                    h_i = h.eval(feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0, learning_rate_: learning_rate})
                    cost.append(np.square(h_i))
                    h1.append(h_i)

                rmse_train = np.sqrt(np.mean(np.square(h1)))
                
                y_pred = []
                for itr in range(iter_test):
                    x_test_batch, y_test_batch = next(testing_generator)
                    h_i = h.eval(feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0, learning_rate_: learning_rate})
                    t1.append(h_i)
                
                rmse_test = np.sqrt(np.mean(np.square(t1)))
                
                #rmse_train = session.run(RMSE, feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
                rmse_test = session.run(RMSE, feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0})
                plot_x.append(ep)
                plot_y1.append(rmse_train)
                plot_y2.append(rmse_test)
                
                #print(ep)

                time_per_ep = (time.time() - __start)
                time_remaining = ((epochs - ep) * time_per_ep) / 3600
                print("LSTM", "epoch:", ep, "RMSE-train:", rmse_train, "RMSE-test", rmse_test, "lr", learning_rate,
                      "\ttime/epoch:", round(time_per_ep, 2), "\ttime_remaining: ",
                      int(time_remaining), " hr:", round((time_remaining % 1) * 60, 1), " min", "\ttime_stamp: ",
                      datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S"))
                __start = time.time()

                if ep % 10 == 0 and ep != 0:
                    save_path = saver.save(session, path_checkpoint)
                    if os.path.exists(path_checkpoint + '.meta'):
                        print("Model saved to file: %s" % path_checkpoint)
                    else:
                        print("NOT SAVED!!!", path_checkpoint)

                if ep % 50 == 0 and ep != 0: 
                    plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
                    plt.show()
                        
                if ep % 200 == 0 and ep != 0: 
                    learning_rate = learning_rate / 2


            save_path = saver.save(session, path_checkpoint)
            if os.path.exists(path_checkpoint + '.meta'):
                print("Model saved to file: %s" % path_checkpoint)
            else:
                print("NOT SAVED!!!", path_checkpoint)
            plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
            plt.show()
        else:
            saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)
            if Predict == True:
                print("Prediction for submit...")
                x_predict = Xs
                y_predict = np.zeros((Xs.shape[0],Xs.shape[1]))

                predict_generator = batch_generator(x_predict, y_predict, batch_size, sequence_length,
                                                       online=True, online_shift=shift)

                full_prediction = []

                iteration = int(x_predict.shape[0] / shift)
                #print("iteration: %i, predgen %s" % (iteration, predict_generator))
                print("#of validation points:", x_predict.shape[0], "#datapoints covers from minibatch:",
                      batch_size * sequence_length, "iterations/epoch", iteration)

                for itr in range(iteration):
                    x_validate_batch, y_validate_batch = next(predict_generator)
                    #print (itr)
                    __y_pred = session.run(output, feed_dict={X: x_validate_batch, Y: y_validate_batch, keep_prob: 1.0})
                    
                    for i in range(batch_size):
                        full_prediction.append(__y_pred[i*sequence_length])
                    #print(__y_pred.shape)
                    
                full_prediction = np.array(full_prediction)
                full_prediction = full_prediction.ravel()
            


Model restored from file: ./save/save_lstm/lstm_2_layers
Training set MSE
No epoches:  1000 No itr:  20
LSTM epoch: 0 RMSE-train: 0.9299958 RMSE-test 0.6088169 lr 0.0004 	time/epoch: 4.94 	time_remaining:  1  hr: 22.3  min 	time_stamp:  2018.11.15-11:35:38
LSTM epoch: 1 RMSE-train: 0.92781144 RMSE-test 0.6102699 lr 0.0004 	time/epoch: 3.93 	time_remaining:  1  hr: 5.4  min 	time_stamp:  2018.11.15-11:35:42
LSTM epoch: 2 RMSE-train: 0.9270182 RMSE-test 0.612442 lr 0.0004 	time/epoch: 3.88 	time_remaining:  1  hr: 4.5  min 	time_stamp:  2018.11.15-11:35:46
LSTM epoch: 3 RMSE-train: 0.92648226 RMSE-test 0.6137757 lr 0.0004 	time/epoch: 3.84 	time_remaining:  1  hr: 3.8  min 	time_stamp:  2018.11.15-11:35:50
LSTM epoch: 4 RMSE-train: 0.9260904 RMSE-test 0.61510617 lr 0.0004 	time/epoch: 3.93 	time_remaining:  1  hr: 5.2  min 	time_stamp:  2018.11.15-11:35:54
LSTM epoch: 5 RMSE-train: 0.9253828 RMSE-test 0.61652434 lr 0.0004 	time/epoch: 3.85 	time_remaining:  1  hr: 3.8  min 	time_stamp:  

In [ ]:
x_validate_batch.shape

In [ ]:
full_prediction

In [ ]:
full_prediction.shape

In [ ]:
shft = x_train.shape[0]+x_test.shape[0]+2*sequence_length

In [ ]:
mean_y

In [ ]:
y_submit = full_prediction*std_y+mean_y
sub_file = pd.DataFrame()
sub_file["timestamp"] = submit_X["timestamp"][shft:]
sub_file["target"] = y_submit
sub_file.to_csv('submit2.csv', index=False)

In [ ]:
# LB 0.4705